In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, Tokenizer, VectorAssembler
import pyspark.sql.functions as F
from pyspark.sql.window import Window
# isnan, when, count, datediff, mean, lag, 
# col, month, year, weekofyear, date_format, dayofmonth, avg, stddev, desc, sum

In [ ]:
import os
pwd = os.getcwd()
exploration_abs_path = f"{pwd}/exploration.ipynb"
exploration_abs_path

In [ ]:
%run {pwd + "/exploration.ipynb"}

In [ ]:
spark = create_spark_session("Spark_Application_Name")

# load data
all_datasets = []
for f in ['AMAZON.csv', 'APPLE.csv', 'FACEBOOK.csv', 'GOOGLE.csv',
            'MICROSOFT.csv', 'TESLA.csv', 'ZOOM.csv']:
    print(f"{f}:")
    df = load_data(spark, 'stocks_data/' + f)
    all_datasets.append(df)

Input:  
    - 4xHigh  
    - 4xOpen  
    - 4xClose  
    - 4xLow  
    - RSI (not implemented)  
    - Volatilité  
    - Momentum  
    - Upside/Downside  
    - IMI  
    - MFI  
  
  
régression polynomiale  
régression quantile  
réseau de neurones récurrent  

In [ ]:
# create a window to get lagged columns
win = Window.partitionBy('Year').orderBy('Date')

In [ ]:
# create lagged columns (High, Low, Open and Close offseted by up to four)
df_with_lag = df.withColumn('Year', F.year('Date'))
for i in range(1, 4):
    for s in ["High", "Low", "Open", "Close"]:
        df_with_lag = df_with_lag.withColumn(s + str(i), F.lag(F.col(s), i).over(win))

In [ ]:
s = 'Date'
i = 3
df_with_lag = df_with_lag.withColumn(s + str(i), F.lag(F.col(s), i).over(win))

In [ ]:
df_with_lag.select(['Close', 'Close1', 'Close2', 'Close3']).show()

In [ ]:
format_date = "%Y-%m-%d %H:%M:%S"
vol = compute_volatility(df_with_lag)
vol.select(['Volatility', 'Close', 'Date']).show()

In [ ]:
# add momentum to the dataframe
df_with_momentum = compute_momentum(df_with_lag).withColumnRenamed('daily_average', 'Momentum')

In [ ]:
# add label (the next Close value)
df_labeled = df_with_momentum.withColumn('label', F.lag(F.col('Close'), -1).over(win))

In [ ]:
df_labeled.toDF(*df_labeled.columns)

In [ ]:
# all the features given to the model
features = [
    "High", "High1", "High2", "High3",
    "Low", "Low1", "Low2", "Low3",
    "Open", "Open1", "Open2", "Open3",
    "Close", "Close1", "Close2", "Close3",
    "Momentum"
]

# drop columns containing null values
final_df = df_labeled.select(features + ['label']).fillna(0)
final_df.show()

In [ ]:
# vectorise features
assembler = VectorAssembler(
    inputCols=  #features,
    [
        "High",
        "Low",
        "Open",
        "Close",
    ],
    outputCol="features"
)

# create model type
lr = LinearRegression()

# create the pipeline that will create the model
pipeline = Pipeline(stages=[assembler, lr])

In [ ]:
#df_labeled = df_with_lag.withColumn('label', F.col('Close'))

In [ ]:
final_df

In [ ]:
# Fit the pipeline to training documents.
model = pipeline.fit(final_df)

In [ ]:
# test the model
predDF = model.transform(df)

In [ ]:
predDF.show()

In [ ]:
df_labeled.select(['Open', 'Close', 'label']).show()